In [23]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [24]:
transform= transforms.ToTensor()

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()            
        self.conv1=nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.pool=nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2=nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5)
        self.conv3=nn.Conv2d(in_channels=32,out_channels= 64, kernel_size=3)
        self.fc1=nn.Linear(in_features=1*1*64, out_features=120)
        self.fc2=nn.Linear(in_features=120, out_features=84)
        self.fc3=nn.Linear(in_features=84, out_features=10)
        
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=self.pool(F.relu(self.conv3(x))) 
        x = x.view(-1, 64 * 1 * 1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
       
net=Net()
net

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=64, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [26]:
import torch.optim as optim
criterion= nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(), lr=0.001)

In [27]:
for epoch in range(2):
    running_loss=0.0
    for i, data in enumerate(train_dataloader,0):
        images, labels=data
        
        optimizer.zero_grad()
        
        # forward 
        outputs= net(images)
       
        loss= criterion(outputs, labels)
    
        loss.backward()
        # update the weights
        optimizer.step()
        #calculate loss
        running_loss+=loss.item()
        if i%1875==1874:
            print('epoch %d - image count %5d - Loss %.3f' % (epoch +1, i+1, running_loss/1875))      
            running_loss = 0.0


#evaluate on test dataset
correct=0
total=0

with torch.no_grad():
 
    for data in test_dataloader:
        images. lables=data
        outputs= net(images)
        _,pred= torch.max(outputs,1)
        total+=labels.size(0)
        correct += (pred == labels).sum().item()
    
print("Eval accuracy using PyTorch is %.2f" %((100 * (correct / total))))

Eval accuracy using PyTorch is 100.00
